<a href="https://colab.research.google.com/github/uomah/Graduation_Project/blob/main/%EA%B3%A0%EB%A0%B9%EC%9E%90_%EA%B5%90%ED%86%B5%EC%82%AC%EA%B3%A0_%EC%B7%A8%EC%95%BD_%EC%A7%80%EC%A0%90_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **고령자 대상 보호구역 설정**

# 분석전 미리 설정해야할 것들

* 한글적용을 위해 글꼴 다운로드

***!! 주의 !!*** 
아래 첫 코드를 실행시킨후, 상단메뉴의  **런타임 -> 런타임 다시시작**을 하여 코드를 돌릴것. 그래야지만 한글이 적용됨

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

*  드라이브와의 연결

In [ ]:

import os
from google.colab import drive
drive.mount("/gdrive",force_remount=True)


In [ ]:
cd /gdrive/MyDrive/JJS/0512JJS/Data_seoul/

### **코드 들어가기 앞서 본프로젝트에 대한 전반적인 설명:**



고령자(60세 이상) 교통사고에 유의미한 영향을 미치는 요인을 분석하기 위해 **14-19년도 사건사고(사망및사고포함)데이터**를 사용했다. 


이를 사망사고(1)와 비사망사고(0)로 분류하여 여러 분류기법을 사용하여 분석했으며, 특성중요도를 바탕으로 고령자의 사망사고에 보다 더 의미있는 영향을 끼치는 요인을 뽑아 냈다. ***// 여기까지가 1번***

그리고 다시, 위치기반의 **12-18사망사고 데이터**를 가져왔고, 앞서 분석해낸 특성을 기준으로 위치를 필터링해서 지도에 표시해본다.

이후, 한번이라도 사망사고가 난 위치를 기준으로 반경 500m내에 고령보행자사고다발지역으로 분류되는 좌표계의 갯수와, 사망사고 갯수를 각각 세어 칼럼에 추가한다.

반경 500m 이내에 **고령보행자사고다발지역**의 좌표개수는 0인 위치들만 따로 출력한다. 이때 엑셀파일을 반경 500m이내의 **사망사고 발생 갯수**를 기준으로 내림차순하여 등한시 되었는 보행사교통사고소외 지역을 뽑아내 보도록 한다. ***//여기까지가 2번***


아래에서는 각각의 작업에 대해 하나씩 논해볼 것이다.

# ***1. 고령자 교통사고에 영향을 미치는 요인 ( 중요한 요인은 무엇일까? )***


> **데이터를 불러오고 다듬는 과정**







* 14-19년도 사건사고 파일을 가져온다









In [ ]:
#파일 가져옴
import pandas as pd
import numpy as np
df_accident=pd.read_csv('14_19_accident_60up.csv', encoding='euc-kr')
df_accident.info()

* 사고유형 대분류를 확인하여 차대사람의 경우만 취한다

In [ ]:
print(df_accident['사고유형_대분류'].value_counts())

In [ ]:
#차대사람일 경우만 분석, 나머지 삭제
df_accident = df_accident[df_accident['사고유형_대분류'] != '차대차' ]
print(df_accident['사고유형_대분류'].value_counts())

* 데이터 전처리를 한다. 
    이때 발생일, 발생지_시도(모두 서울), 피해당사자종별(모두 보행자), 사고유형 대/중분류( 소분류 존재) 는 삭제한다.

In [ ]:
df_accident.drop(columns=['발생일'], inplace= True)
df_accident.drop(columns=['발생지_시도'], inplace= True)
df_accident.drop(columns=['사고유형_대분류'], inplace= True)
df_accident.drop(columns=['사고유형_중분류'], inplace= True)
df_accident.drop(columns=['피해당사자종별'], inplace= True)
df_accident.columns

* 가해자신체상해정도역시 삭제한다. 왜냐하면 사고발생에 미치는 영향이 아닌 사망자수 등과같이 사고발생후 결과로 해석되기 때문이다. 마찬가지로 피해자 신체상해정도도 삭제한다.

In [ ]:
df_accident.drop(columns=['가해자신체상해정도'], inplace= True)
df_accident.drop(columns=['피해자신체상해정도'], inplace= True)

* 각종 칼럼들의 기타, 또는 불명 데이터를 제거 한다.

In [ ]:
#기타 불명 삭제
df_accident = df_accident[df_accident['사고유형'] != "기타"]
df_accident = df_accident[df_accident['가해자법규위반'] != "기타"]
df_accident = df_accident[df_accident['도로형태_대분류'] != "기타" ]
df_accident = df_accident[df_accident['도로형태_대분류'] != "기타/불명" ]
df_accident = df_accident[df_accident['도로형태_대분류'] != "불명" ]
df_accident = df_accident[df_accident['기상상태'] != "기타/불명" ]
df_accident = df_accident[df_accident['가해자성별'] != "기타불명" ]


*   가해당사자 종별 (가해차량)의 값을 확인하고, 6가지 경우로 분류하여 줄인다.





In [ ]:
#가해차량종류를 모두 확인
print(df_accident['가해당사자종별'].value_counts())

In [ ]:
#기타/불명값과, 분류하기 애매한 개인형 이동수단은 삭제함
df_accident = df_accident[df_accident['가해당사자종별'] != "기타"]
df_accident = df_accident[df_accident['가해당사자종별'] != "불명"]
df_accident = df_accident[df_accident['가해당사자종별'] != "개인형이동수단(PM)"]
#replace함수를 사용하여 특정 열에 해당하는 값 변경
df_accident.replace({'가해당사자종별':'사륜오토바이(ATV)'},{'가해당사자종별':'이륜차'},inplace=True)
df_accident.replace({'가해당사자종별':'원동기장치자전거'},{'가해당사자종별':'이륜차'},inplace=True)
df_accident.replace({'가해당사자종별':'건설기계'},{'가해당사자종별':'기계차'},inplace=True)
df_accident.replace({'가해당사자종별':'특수차'},{'가해당사자종별':'기계차'},inplace=True)
df_accident.replace({'가해당사자종별':'농기계'},{'가해당사자종별':'기계차'},inplace=True)
print(df_accident['가해당사자종별'].value_counts())

*  노면상태를 확인후, 건조 < 젖음 < 적설 < 서리 순으로 0부터 3까지의 번호를 부여하여 연속형변수로 만든다. 

In [ ]:
print(df_accident['노면상태'].value_counts())

In [ ]:
#기타는 완전 삭제
df_accident = df_accident[df_accident['노면상태'] != "기타"]
#replace함수를 사용하여 특정 열에 해당하는 값 변경
df_accident.replace({'노면상태':'건조'},{'노면상태':0},inplace=True)
df_accident.replace({'노면상태':'젖음/습기'},{'노면상태':1},inplace=True)
df_accident.replace({'노면상태':'적설'},{'노면상태':2},inplace=True)
df_accident.replace({'노면상태':'서리/결빙'},{'노면상태':3},inplace=True)

print(df_accident['노면상태'].value_counts())

* 일차적인 데이터 전처리완료, 어지러져 있는 index를 다시 부여해줌 

In [ ]:
df_accident = df_accident.reset_index()
df_accident

In [ ]:
#추가된 index행 삭제
df_accident.drop(columns=['index'], inplace= True)
df_accident

> 일반교통사고 대 사망교통사고 로 분류, 타겟변수를 생성함

* 사망자수에 관한 열 정보를 확인

In [ ]:
print(df_accident['사망자수'].value_counts())

*  사망교통사고여부 칼럼 생성후 사망자수 컬럼복사, 그리고 사망자수가 1회이상인 케이스를 모두 1로 분류한다.

In [ ]:
df_accident['사망교통사고여부'] = df_accident['사망자수']
df_accident.replace({'사망교통사고여부':2},{'사망교통사고여부':1},inplace=True)
print(df_accident['사망교통사고여부'].value_counts())



> 범주형데이터를 각각의 0/1로 표시되는 데이터로 바꾸도록 원-핫 인코딩 실시



*  df_one_hot 변수에 복사후 데이터 확인

In [ ]:
df_one_hot = df_accident
display(df_one_hot.head())

*  컬럼 갯수 확인을 통해 원핫인코딩이 제대로 됬음을 확인

In [ ]:
print("원본 특성:\n", list(df_one_hot.columns),"\n")
df_one_hot = pd.get_dummies(df_one_hot)
print("이후 특성:\n", list(df_one_hot.columns),"\n")
display(df_one_hot.head())


> **본격적인 분석을 시작해보자!**





본격적인 분류를 시작해본다.

In [ ]:
!pip install mglearn

* 독립변수(X)와 종속변수(y)를 나누어 저장

In [ ]:
feature_cols = ['사고유형_길가장자리구역통행중', '사고유형_보도통행중', 
      '사고유형_차도통행중', '사고유형_횡단중', '가해자법규위반_과속',
       '가해자법규위반_교차로 통행방법 위반', '가해자법규위반_보행자 보호의무 위반', '가해자법규위반_신호위반',
       '가해자법규위반_안전거리 미확보', '가해자법규위반_안전운전 의무 불이행', '가해자법규위반_중앙선 침범',
       '도로형태_고가도로위', '도로형태_교량위', '도로형태_교차로내', '도로형태_교차로부근', '도로형태_교차로횡단보도내',
       '도로형태_기타단일로', '도로형태_지하차도(도로)내', '도로형태_횡단보도부근', '도로형태_횡단보도상',
       '노면상태_대분류_비포장', '노면상태_대분류_포장', '기상상태_눈', '기상상태_맑음', '기상상태_비', '기상상태_안개',
       '기상상태_흐림', '가해당사자종별_기계차', '가해당사자종별_승용차', '가해당사자종별_승합차', '가해당사자종별_이륜차',
       '가해당사자종별_자전거', '가해당사자종별_화물차', '가해자성별_남', '가해자성별_여']
X = df_one_hot[feature_cols]
y = df_one_hot['사망교통사고여부']



> 의사결정트리모델 사용





*   train set과 test set으로 나누어 저장


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics 
import seaborn as sns
X_train, X_test, y_train, y_test =train_test_split(X, y, random_state=0)

* tree의 최대 깊이를 다르게 해가며 훈련세트와 테스트세트의 정확도를 확인

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state = 0)
tree.fit(X_train,y_train)

print("훈련세트 정확도:{:.4f}".format(tree.score(X_train,y_train)))
print("테스트세트 정확도:{:.4f}".format(tree.score(X_test,y_test)))

In [ ]:
tree = DecisionTreeClassifier(max_depth = 10 , random_state = 0)
tree.fit(X_train,y_train)

print("훈련세트 정확도:{:.4f}".format(tree.score(X_train,y_train)))
print("테스트세트 정확도:{:.4f}".format(tree.score(X_test,y_test)))

In [ ]:
tree = DecisionTreeClassifier(max_depth = 15 , random_state = 0)
tree.fit(X_train,y_train)

print("훈련세트 정확도:{:.4f}".format(tree.score(X_train,y_train)))
print("테스트세트 정확도:{:.4f}".format(tree.score(X_test,y_test)))

*  모델을 실제 의사결정트리로 시각화 함

In [ ]:
#트리의 결과값을 문서로 출력후 
from sklearn.tree import export_graphviz
export_graphviz(tree,out_file="tree.dot",feature_names=feature_cols,impurity=False,filled=True)

In [ ]:
#시각화 툴을 사용하여 불러와서 시각화 시킴
import graphviz

with open("tree.dot") as f:
  dot_graph = f.read()
display(graphviz.Source(dot_graph))

*  각각 독립변수의 특성중요도를 시각화 시켜봄

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
#특성중요도 시각화
feature_importance = tree.feature_importances_
df_fi = pd.DataFrame({'columns':X.columns, 'importances':feature_importance})
df_fi = df_fi[df_fi['importances'] > 0] # 중요도, 즉 importance가 0이상인 것만 
df_fi = df_fi.sort_values(by=['importances'], ascending=False)

fig = plt.figure(figsize=(15,7)) #차트 크기 설정함수
ax = sns.barplot(df_fi['columns'], df_fi['importances'])
ax.set_xticklabels(df_fi['columns'], rotation=80, fontsize=13)
plt.tight_layout()
plt.show()



> 랜덤포레스트 모델의 사용



* 마찬가지로 train set과 test set을 나누어 사용함

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics 
import seaborn as sns
X_train, X_test, y_train, y_test =train_test_split(X, y, random_state=0)

*  위의 일반 tree와 마찬가지로 최대깊이 15의 트리를 생성

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(max_depth = 15 ,random_state = 0)
forest.fit(X_train, y_train)

print("훈련세트 정확도:{:.4f}".format(forest.score(X_train,y_train)))
print("테스트세트 정확도:{:.4f}".format(forest.score(X_test,y_test)))

* 바로 해당 모델의 특성중요도를 시각화 시켰다.

In [ ]:
#특성중요도 시각화
feature_importance = forest.feature_importances_
df_fi = pd.DataFrame({'columns':X.columns, 'importances':feature_importance})
df_fi = df_fi[df_fi['importances'] > 0] # 중요도, 즉 importance가 0이상인 것만 
df_fi = df_fi.sort_values(by=['importances'], ascending=False)

fig = plt.figure(figsize=(15,7)) #차트 크기 설정함수
ax = sns.barplot(df_fi['columns'], df_fi['importances'])
ax.set_xticklabels(df_fi['columns'], rotation=80, fontsize=13)
plt.tight_layout()
plt.show()

# ***2. 위치 데이터 기반, 실제 노인교통사고 취약지 도출***


* 위치 분석을 위해, 위치기반 사망사고 데이터를 가져옴 

In [ ]:
#파일 가져옴
import pandas as pd
import numpy as np
df_location =pd.read_csv('12_18_death_60up.csv', encoding='euc-kr')



> **교통사건사고데이터와 유사하게 전처리**




* 사고유형 대분류를 확인후 차대 사람 사고만 남겨둠

In [ ]:
print(df_location['사고유형_대분류'].value_counts())

In [ ]:
#차대사람일 경우만 분석, 나머지 삭제
df_location = df_location[df_location['사고유형_대분류'] != '차대차' ]
print(df_location['사고유형_대분류'].value_counts())

* 아까와 마찬가지고 필요없다 판단되는 칼럼 삭제

In [ ]:
df_location.drop(columns=['발생일'], inplace= True)
df_location.drop(columns=['사고내용'], inplace= True)
df_location.drop(columns=['사고유형_대분류'], inplace= True)
df_location.drop(columns=['사고유형_중분류'], inplace= True)
df_location.drop(columns=['피해자차종'], inplace= True)
df_location.columns

* 가해자차종(=가해당사자종별)을 단순화



In [ ]:
print(df_location['가해자차종'].value_counts())

In [ ]:
#기타/불명값과, 분류하기 애매한 개인형 이동수단은 삭제함
df_location = df_location[df_location['가해자차종'] != "불명"]
#replace함수를 사용하여 특정 열에 해당하는 값 변경
df_location.replace({'가해자차종':'원동기장치자전거'},{'가해자차종':'이륜차'},inplace=True)
df_location.replace({'가해자차종':'자전거'},{'가해자차종':'이륜차'},inplace=True)
df_location.replace({'가해자차종':'건설기계'},{'가해자차종':'기계차'},inplace=True)
df_location.replace({'가해자차종':'특수차'},{'가해자차종':'기계차'},inplace=True)
print(df_location['가해자차종'].value_counts())


> **'반경 500m내 고령자 교통사고 사망사고 수' 카운트 하여 새로운 칼럼으로 생성**



In [ ]:
#아래 함수 사용을 위해 임시로 위도 경도를 영어로 표기
df_location = df_location.rename(columns={'위도':'lat','경도':'lot'})

* count_func이라는 함수 정의 (df_location 파일을 기준으로 input한 파일의 위경도 값을 사용하여 카운트한후 list에 저장시킴)

In [ ]:


def count_func(loc_data, list_frame):
    for lng_i, lat_i in zip(df_location.lot, df_location.lat) :
        
        count = 0
        for t_lng_i, t_lat_i in zip(loc_data.lot, loc_data.lat):  
            if ((lng_i-t_lng_i)**2 + (lat_i - t_lat_i)**2)**(1/2) <= 0.0045: #0.0035, 0.003, 0.002, 0.001 
                count += 1    
                print(t_lng_i, t_lat_i )
        list_frame.append(count)       



In [ ]:
count_lit = []
count_func(df_location,count_lit)

* df_location에 새로운 칼럼 '반경 500m내 사고수' 생성

In [ ]:
df_location.loc[:,'acc_num'] = count_lit
df_location = df_location.rename(columns={'acc_num':'반경500m내사고수'})



> **'반경 500m내 고령자 교통사고 사고다발지역 수' 카운트 하여 새로운 칼럼으로 생성**




* 새로운 데이터인 12_19년도 고령자 사고 다발지 파일 가져옴, 이후 필요한 칼럼만 가져와 accident_area변수에 저장

In [ ]:
#파일 가져옴
import pandas as pd
import numpy as np
accident_area =pd.read_csv('12_19_oldman.csv', encoding='euc-kr')
accident_area = accident_area[['시도시군구명', '지점명', '발생건수', '사상자수',
                               '사망자수', '중상자수', '경상자수', '부상신고자수', '경도', '위도']]

* 카운트 하기위해 임시로 위도, 경도를 lat lot으로 변경

In [ ]:
accident_area = accident_area.rename(columns={'위도':'lat','경도':'lot'})

* 임시 list 생성후 아까 만들었던 count_func 함수 사용. 이떄 input값은 사고다발지 정보

In [ ]:
count_lit2 = []
count_func(accident_area,count_lit2)

* 새로운 칼럼 '반경500m내 사고다발지' 생성후 저장

In [ ]:
df_location.loc[:,'load_num'] = count_lit2
df_location
df_location = df_location.rename(columns={'load_num':'반경500m내사고다발지'})

*  혼동방지를 위해 df_location의 칼럼명을 다시 위도 - 경도로 변경

In [ ]:
df_location = df_location.rename(columns={'lat':'위도','lot':'경도'})
accident_area = accident_area.rename(columns={'lat':'위도','lot':'경도'})

여기서 부터  앞서 찾아낸 특성들중에, 중요도가 높으며 노인사망교통사고에 관련도가 깊다고 판단되는 변수 3가지를 **각각 지도에 띄워보며** 노인의 교통사고 **취약지점**이지만 **사고다발지임이 밝혀지지 않은 지역**들을 찾아보기로 했다.



> **가해자 차종 - 기계차 / 가해자법규위반- 과속 / 도로형태 - 횡단보도상 / 사고유형 - 횡단중**



* 첫번째 **가해자 차종이 기계차**인경우를 **파란색**으로 지도에 표시, 반경내 사고수와 사고다발지수를 확인할 수 있다. 

In [ ]:
# 50개 데이터만 남음
df_car = df_location[df_location['가해자차종'] == '기계차' ]

In [ ]:
import json
import folium
map_osm = folium.Map(location=[37.5642135, 127.0016985], zoom_start=11)
for index, row in df_car.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                              icon = folium.Icon(icon_color='blue')).add_to(map_osm)
map_osm

In [ ]:
# 22개 데이터만 남음
df_attack = df_location[df_location['가해자법규위반'] == '과속' ]
df_attack.info()

In [ ]:
import json
import folium

for index, row in df_attack.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                  icon = folium.Icon(color = 'red', icon_color='red')).add_to(map_osm)
map_osm

In [ ]:
# 73개 데이터만 남음
df_load = df_location[df_location['도로형태'] == '횡단보도상' ]
df_load.info()

In [ ]:
import json
import folium

for index, row in df_load.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                  icon = folium.Icon(color = 'green', icon_color='green')).add_to(map_osm)
map_osm

In [ ]:
# 73개 데이터만 남음
df_acc = df_location[df_location['사고유형'] == '횡단중' ]
df_acc.info()

In [ ]:

for index, row in df_acc.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                  icon = folium.Icon(color = 'pink', icon_color='pink')).add_to(map_osm)

map_osm

> **사고다발지역 정보를 바탕으로 한번에 지도에 띄운모습**

In [ ]:
accident_area = accident_area[['시도시군구명', '지점명', '발생건수', '사상자수',
                               '사망자수', '중상자수', '경상자수', '부상신고자수', '경도', '위도']]

In [ ]:
import json
import folium
map_acc = folium.Map(location=[37.5642135, 127.0016985], zoom_start=10)
for index, row in accident_area.iterrows():
    location = (row['위도'], row['경도'])
    folium.CircleMarker(location,icon = folium.Icon(color = 'gray')).add_to(map_acc)

map_acc

In [ ]:
import json
import folium
map_final = folium.Map(location=[37.5642135, 127.0016985], zoom_start=10)
for index, row in accident_area.iterrows():
    location = (row['위도'], row['경도'])
    folium.CircleMarker(location,icon = folium.Icon(color = 'gray')).add_to(map_final)

for index, row in df_acc.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                  icon = folium.Icon(color = 'pink', icon_color='pink')).add_to(map_final)

for index, row in df_load.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                  icon = folium.Icon(color = 'green', icon_color='green')).add_to(map_final)

for index, row in df_attack.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])), 
                  icon = folium.Icon(color = 'red', icon_color='red')).add_to(map_final)

for index, row in df_car.iterrows():
    location = (row['위도'], row['경도'])
    folium.Marker(location, tooltip = ('<b> 반경500m내 사고수: </b> {num_a}명<br>' 
                                      '<b> 반경 500m내 사고다발지: </b> {num_b}개<br>'
                                      ).format(
                                              num_a = int(row['반경500m내사고수']),
                                              num_b = int(row['반경500m내사고다발지'])),   
                  icon = folium.Icon(icon_color='blue')).add_to(map_final)

map_final

In [ ]:
df_car_sort = df_car[df_car['반경500m내사고다발지'] == 0 ]
df_car_sort.sort_values(by = '반경500m내사고수',ascending=False)
df_car_sort.to_excel('df_car_sort.xlsx')

In [ ]:
df_attack_sort = df_attack[df_attack['반경500m내사고다발지'] == 0 ]
df_attack_sort.sort_values(by = '반경500m내사고수',ascending=False)
df_attack_sort.to_excel('df_attack_sort.xlsx')

In [ ]:
df_load_sort = df_load[df_load['반경500m내사고다발지'] == 0 ]
df_load_sort.sort_values(by = '반경500m내사고수',ascending=False)
df_load_sort.to_excel('df_load_sort.xlsx')

In [ ]:
df_acc_sort = df_acc[df_acc['반경500m내사고다발지'] == 0 ]
df_acc_sort.sort_values(by = '반경500m내사고수',ascending=False)
df_acc_sort.to_excel('df_acc_sort.xlsx')